### Dependencies 

In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
from starfish import StructuredLLM, data_factory
from starfish.common.env_loader import load_env_file
from starfish.llm.utils import merge_structured_outputs
load_env_file()

2025-04-12 09:40:02 | INFO     | starfish.common.env_loader | env_loader.py:52 | Loaded 9 environment variables from /Users/zhengisamazing/1.python_dir/starfish/.env


True

In [3]:
### Mock LLM call
import random
import asyncio

async def mock_llm_call(city_name, num_records_per_city, fail_rate=0.05, sleep_time=0.01):
    # Simulate a slight delay (optional, feels more async-realistic)
    await asyncio.sleep(sleep_time)

    # 5% chance of failure
    if random.random() < fail_rate:
        print(f"  {city_name}: Failed!") ## For debugging
        raise ValueError(f"Mock LLM failed to process city: {city_name}")
    
    print(f"{city_name}: Successfully processed!") ## For debugging

    result = [f"{city_name}_{random.randint(1, 5)}" for _ in range(num_records_per_city)]
    return result

Test Case 1:  ✅

Settings:
- ✅ `Input Data`:  data + broadcast variable`
- ✅ `Decorator`: controlled concurrency
- ✅ `Retry Logic`: Try if failed 

In [4]:

@data_factory(max_concurrency=5)
async def test1(city_name, num_records_per_city, fail_rate = 0.5, sleep_time = 1):
    return await mock_llm_call(city_name, num_records_per_city, fail_rate = fail_rate, sleep_time = sleep_time)

test1.run(data = [
    {'city_name': '1. New York'},
    {'city_name': '2. Los Angeles'},
    {'city_name': '3. Chicago'},
    {'city_name': '4. Houston'},
    {'city_name': '5. Miami'}
], num_records_per_city=5)

2025-04-12 09:40:02 | INFO     | starfish.utils.data_factory | data_factory.py:180 | 
2. Creating master job...
2025-04-12 09:40:02 | INFO     | starfish.utils.job_manager | job_manager.py:154 | No task to run, waiting for task to complete
2025-04-12 09:40:03 | INFO     | starfish.utils.job_manager | job_manager.py:154 | No task to run, waiting for task to complete
1. New York: Successfully processed!
  2. Los Angeles: Failed!
3. Chicago: Successfully processed!
  4. Houston: Failed!
5. Miami: Successfully processed!
2025-04-12 09:40:04 | INFO     | starfish.utils.job_manager | job_manager.py:154 | No task to run, waiting for task to complete
2025-04-12 09:40:05 | INFO     | starfish.utils.job_manager | job_manager.py:154 | No task to run, waiting for task to complete
2. Los Angeles: Successfully processed!
  4. Houston: Failed!
2025-04-12 09:40:06 | INFO     | starfish.utils.job_manager | job_manager.py:154 | No task to run, waiting for task to complete
2025-04-12 09:40:06 | ERROR    

/Users/zhengisamazing/Library/Caches/pypoetry/virtualenvs/starfish-T7IInzTH-py3.11/lib/python3.11/site-packages/ric
h/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

[['1. New York_3',
  '1. New York_4',
  '1. New York_4',
  '1. New York_3',
  '1. New York_3'],
 ['3. Chicago_2',
  '3. Chicago_3',
  '3. Chicago_2',
  '3. Chicago_1',
  '3. Chicago_2'],
 ['5. Miami_4', '5. Miami_3', '5. Miami_2', '5. Miami_5', '5. Miami_3'],
 ['2. Los Angeles_3',
  '2. Los Angeles_3',
  '2. Los Angeles_4',
  '2. Los Angeles_2',
  '2. Los Angeles_5'],
 ['4. Houston_3',
  '4. Houston_1',
  '4. Houston_1',
  '4. Houston_4',
  '4. Houston_2']]

Test Case 2: ❌

Expected behavior: Data should be optional - use can just pass kwargs variables 

Settings:
- ❌ `Input Data`:  kwargs list + broadcast variable`



In [5]:

@data_factory(max_concurrency=2)
async def test1(city_name, num_records_per_city, fail_rate = 0.5, sleep_time = 1):
    return await mock_llm_call(city_name, num_records_per_city, fail_rate = fail_rate, sleep_time = sleep_time)

test1.run(city_name = ["1. New York", "2. Los Angeles", "3. Chicago", "4. Houston", "5. Miami"], num_records_per_city=5)

2025-04-12 09:40:28 | INFO     | starfish.utils.data_factory | data_factory.py:180 | 
2. Creating master job...
1. New York: Successfully processed!
2. Los Angeles: Successfully processed!
3. Chicago: Successfully processed!
  4. Houston: Failed!
2025-04-12 09:40:30 | INFO     | starfish.utils.job_manager | job_manager.py:154 | No task to run, waiting for task to complete
2025-04-12 09:40:31 | INFO     | starfish.utils.job_manager | job_manager.py:154 | No task to run, waiting for task to complete
5. Miami: Successfully processed!
2025-04-12 09:40:32 | INFO     | starfish.utils.job_manager | job_manager.py:154 | No task to run, waiting for task to complete
4. Houston: Successfully processed!
2025-04-12 09:40:33 | INFO     | starfish.utils.data_factory | data_factory.py:258 | Master job 7706e3e3-3b31-4f6c-9303-9eb0eccec022 as completed


[['1. New York_2',
  '1. New York_4',
  '1. New York_2',
  '1. New York_1',
  '1. New York_3'],
 ['2. Los Angeles_5',
  '2. Los Angeles_2',
  '2. Los Angeles_5',
  '2. Los Angeles_4',
  '2. Los Angeles_2'],
 ['3. Chicago_5',
  '3. Chicago_3',
  '3. Chicago_5',
  '3. Chicago_1',
  '3. Chicago_4'],
 ['5. Miami_1', '5. Miami_3', '5. Miami_3', '5. Miami_3', '5. Miami_3'],
 ['4. Houston_3',
  '4. Houston_1',
  '4. Houston_4',
  '4. Houston_1',
  '4. Houston_1']]

Test Case 3:  ❌

Description: Test if the system handles failures gracefully, ensuring it doesnt run indefinitely when the failure rate is 100%

Expected behavior: It should stop after it was not able to process it after cetain times 

Settings:
- ❌ `Handle failures`: Not infinite loop

In [6]:

@data_factory(max_concurrency=2)
async def test1(city_name, num_records_per_city, fail_rate = 1, sleep_time = 0.05):
    return await mock_llm_call(city_name, num_records_per_city, fail_rate = fail_rate, sleep_time = sleep_time)

test1.run(data = [
    {'city_name': '1. New York'},
    {'city_name': '2. Los Angeles'},
    {'city_name': '3. Chicago'},
    {'city_name': '4. Houston'},
    {'city_name': '5. Miami'}
], num_records_per_city=5)

2025-04-12 09:40:37 | INFO     | starfish.utils.data_factory | data_factory.py:180 | 
2. Creating master job...
  1. New York: Failed!
  2. Los Angeles: Failed!
  1. New York: Failed!
  2. Los Angeles: Failed!
2025-04-12 09:40:39 | ERROR    | starfish.utils.task_runner | task_runner.py:30 | Task execution failed after 1 retries
2025-04-12 09:40:39 | ERROR    | starfish.utils.job_manager | job_manager.py:181 | Error running task: Mock LLM failed to process city: 1. New York
2025-04-12 09:40:39 | ERROR    | starfish.utils.task_runner | task_runner.py:30 | Task execution failed after 1 retries
2025-04-12 09:40:39 | ERROR    | starfish.utils.job_manager | job_manager.py:181 | Error running task: Mock LLM failed to process city: 2. Los Angeles
  3. Chicago: Failed!
  4. Houston: Failed!
  3. Chicago: Failed!
  4. Houston: Failed!
2025-04-12 09:40:41 | ERROR    | starfish.utils.task_runner | task_runner.py:30 | Task execution failed after 1 retries
2025-04-12 09:40:41 | ERROR    | starfish.u

KeyboardInterrupt: 

  4. Houston: Failed!
  5. Miami: Failed!
2025-04-12 09:41:30 | ERROR    | starfish.utils.task_runner | task_runner.py:30 | Task execution failed after 1 retries
2025-04-12 09:41:30 | ERROR    | starfish.utils.job_manager | job_manager.py:181 | Error running task: Mock LLM failed to process city: 4. Houston
2025-04-12 09:41:30 | ERROR    | starfish.utils.task_runner | task_runner.py:30 | Task execution failed after 1 retries
2025-04-12 09:41:30 | ERROR    | starfish.utils.job_manager | job_manager.py:181 | Error running task: Mock LLM failed to process city: 5. Miami


Test Case 4:  ✅

Description: Test kwags keyword overide by broacase variables

Settings:
- ✅ `Input Data`:  data + broadcast variable override

In [7]:

@data_factory(max_concurrency=2)
async def test1(city_name, num_records_per_city, fail_rate = 0.1, sleep_time = 0.05):
    return await mock_llm_call(city_name, num_records_per_city, fail_rate = fail_rate, sleep_time = sleep_time)

test1.run(data = [
    {'city_name': '1. New York'},
    {'city_name': '2. Los Angeles'},
], city_name = 'override_city_name', 
num_records_per_city = 1)

2025-04-12 09:41:34 | INFO     | starfish.utils.data_factory | data_factory.py:180 | 
2. Creating master job...
2025-04-12 09:41:34 | INFO     | starfish.utils.job_manager | job_manager.py:154 | No task to run, waiting for task to complete
override_city_name: Successfully processed!
override_city_name: Successfully processed!
2025-04-12 09:41:35 | INFO     | starfish.utils.data_factory | data_factory.py:258 | Master job a4e8e8a4-89a8-4de7-9fe9-29b0e9c2124f as completed


[['override_city_name_5'], ['override_city_name_3']]

Test Case 5:  ✅

Description: Test kwags keyword overide by list variables

Settings:
- ✅ `Input Data`:  data + broadcast variable override

In [8]:
@data_factory(max_concurrency=2)
async def test1(city_name, num_records_per_city, fail_rate = 0.1, sleep_time = 0.05):
    return await mock_llm_call(city_name, num_records_per_city, fail_rate = fail_rate, sleep_time = sleep_time)

test1.run(data = [
    {'city_name': '1. New York'},
    {'city_name': '2. Los Angeles'},
], city_name = ['1. override_city_name', '2. override_city_name'], 
num_records_per_city = 1)

2025-04-12 09:41:36 | INFO     | starfish.utils.data_factory | data_factory.py:180 | 
2. Creating master job...
2025-04-12 09:41:36 | INFO     | starfish.utils.job_manager | job_manager.py:154 | No task to run, waiting for task to complete
1. override_city_name: Successfully processed!
  2. override_city_name: Failed!
2025-04-12 09:41:37 | INFO     | starfish.utils.job_manager | job_manager.py:154 | No task to run, waiting for task to complete
2025-04-12 09:41:38 | INFO     | starfish.utils.job_manager | job_manager.py:154 | No task to run, waiting for task to complete
2. override_city_name: Successfully processed!
2025-04-12 09:41:39 | INFO     | starfish.utils.data_factory | data_factory.py:258 | Master job d5a9f32c-0423-434b-a39e-02384a6cb3a6 as completed


[['1. override_city_name_5'], ['2. override_city_name_3']]

Test Case 6:  ❌

Description: Missing required kwags

Expected behavior: it should raise an error about missing required kwags, somehow it is running indefinitely

Settings:
- ❌ `Missing required num_records_per_city`: 


In [10]:

@data_factory(max_concurrency=2)
async def test1(city_name, num_records_per_city, fail_rate = 0.1, sleep_time = 0.05):
    return await mock_llm_call(city_name, num_records_per_city, fail_rate = fail_rate, sleep_time = sleep_time)

test1.run(data = [
    {'city_name': '1. New York'},
    {'city_name': '2. Los Angeles'},
], city_name = 'override_city_name',  
)

AttributeError: 'DataFactory' object has no attribute 'job_manager'

Test Case 7:  ❌

Description: Pass extra parameters that are not defined in the workflow

Expected behavior: it should raise an error about extra kwargs, somehow it is running indefinitely

Settings:
- ❌ `Add additional parameters'


In [11]:

@data_factory(max_concurrency=2)
async def test1(city_name, num_records_per_city, fail_rate = 0.1, sleep_time = 0.05):
    return await mock_llm_call(city_name, num_records_per_city, fail_rate = fail_rate, sleep_time = sleep_time)

test1.run(data = [
    {'city_name': '1. New York'},
    {'city_name': '2. Los Angeles'},
], num_records_per_city = 1, random_param = 'random_param')

AttributeError: 'DataFactory' object has no attribute 'job_manager'

Test Case 8:  ❌

Description: Test hooks that change the state of the workflow and workflow can access to this data

Expected behavior: it should be able to see the state changed which we do - but it is running in indefinite loop

Settings:
- ❌ `Add additional parameters'


In [13]:

def test_hook(data, SharedState):
    SharedState['variable'] =  f'changed_state - {data}'
    return SharedState


@data_factory(max_concurrency=2, on_record_complete=[test_hook], initial_state_values = {'variable': 'initial_state'})
async def test1(city_name, num_records_per_city, fail_rate = 0.1, sleep_time = 0.05):
    print(f"Checking state: {test1.SharedState['variable']}")
    return await mock_llm_call(city_name, num_records_per_city, fail_rate = fail_rate, sleep_time = sleep_time)

test1.run(data = [
    {'city_name': '1. New York'},
    {'city_name': '2. Los Angeles'},
], num_records_per_city = 1)

2025-04-12 09:42:57 | INFO     | starfish.utils.data_factory | data_factory.py:180 | 
2. Creating master job...
2025-04-12 09:42:57 | INFO     | starfish.utils.job_manager | job_manager.py:154 | No task to run, waiting for task to complete
2025-04-12 09:42:58 | INFO     | starfish.utils.job_manager | job_manager.py:154 | No task to run, waiting for task to complete
2025-04-12 09:42:59 | ERROR    | starfish.utils.task_runner | task_runner.py:30 | Task execution failed after 1 retries
2025-04-12 09:42:59 | ERROR    | starfish.utils.job_manager | job_manager.py:181 | Error running task: 'function' object has no attribute 'SharedState'
2025-04-12 09:42:59 | ERROR    | starfish.utils.task_runner | task_runner.py:30 | Task execution failed after 1 retries
2025-04-12 09:42:59 | ERROR    | starfish.utils.job_manager | job_manager.py:181 | Error running task: 'function' object has no attribute 'SharedState'
2025-04-12 09:42:59 | INFO     | starfish.utils.job_manager | job_manager.py:154 | No ta

/Users/zhengisamazing/Library/Caches/pypoetry/virtualenvs/starfish-T7IInzTH-py3.11/lib/python3.11/site-packages/ric
h/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

KeyboardInterrupt: 